In [166]:
# importy
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence

In [167]:
# načtení a zobrazení souboru a datových typů
data = pd.read_csv("df.csv")
print(data.head())
print(data.dtypes)

  Company    CR    DA     FL     OPM      RT   TAT    DE     Y     PT  \
0   2Q3DU  1.19  0.20   2.21  0.0770    6.66  0.40  0.43  1.95   5.93   
1   46JNq  0.29  0.64  10.14  0.2037   20.04  0.36  6.52  7.26  43.48   
2   4ISOt  2.93  0.04   1.43  0.3055    6.55  0.72  0.06  7.78  18.38   
3   5AWwu  1.03  0.25   3.55  0.0708  146.43  1.80  0.88  6.17   5.15   
4   5hbZR  1.20  0.47   0.00  0.1677   30.92  0.93  0.00  0.00  17.06   

                   Sector  
0             Industrials  
1  Communication Services  
2  Communication Services  
3      Consumer Defensive  
4       Consumer Cyclical  
Company     object
CR         float64
DA         float64
FL         float64
OPM        float64
RT         float64
TAT        float64
DE         float64
Y          float64
PT         float64
Sector      object
dtype: object


# část 1

- Vytvoření korelační matice a vyhodnocení, který ukazatel má největší vliv na indikátor Y akcie

Pro vytvoření korelační matice si vytvořím novou tabulku pouze s číselnými sloupci, protože pandas zápasí s nečíselnými sloupci. 

In [10]:
data_corr = data.drop(columns=["Company", "Sector"])
data_corr.corr()

,CR,DA,FL,OPM,RT,TAT,DE,Y,PT
CR,1.000000,-0.225917,-0.179232,0.284529,-0.151203,-0.123665,-0.141136,-0.018690,-0.040483
DA,-0.225917,1.000000,0.340320,0.015032,0.206620,-0.083691,0.464786,0.164599,-0.120993
FL,-0.179232,0.340320,1.000000,-0.112430,-0.076335,0.156169,0.968612,0.759323,0.032588
OPM,0.284529,0.015032,-0.112430,1.000000,0.051672,-0.298701,-0.050014,0.051308,0.021409
RT,-0.151203,0.206620,-0.076335,0.051672,1.000000,0.206580,-0.065267,-0.046290,0.097708
TAT,-0.123665,-0.083691,0.156169,-0.298701,0.206580,1.000000,0.094869,0.301222,0.017872
DE,-0.141136,0.464786,0.968612,-0.050014,-0.065267,0.094869,1.000000,0.724604,-0.025348
Y,-0.018690,0.164599,0.759323,0.051308,-0.046290,0.301222,0.724604,1.000000,-0.001363
PT,-0.040483,-0.120993,0.032588,0.021409,0.097708,0.017872,-0.025348,-0.001363,1.000000


Největší vliv na indikátor akcie Y má ukazatel FL (finanční páka). Korelační koeficient u něj v korelační matici vyšel nejvyšši, a to 0,76. 

# část 2

- Vytvoření regresního modelu, který bude mít koeficient Y jako vysvětlovanou proměnnou. Do modelu jsou vloženy hodnoty ostatních indikátorů jako vysvětlující proměnné.

In [83]:
formula = "Y ~ CR + DA + FL + OPM + RT + TAT + DE + PT"
mod = smf.ols(formula=formula, data=data)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.635
Method:                 Least Squares   F-statistic:                     22.52
Date:                Fri, 10 Nov 2023   Prob (F-statistic):           1.45e-18
Time:                        19:57:38   Log-Likelihood:                -375.94
No. Observations:                 100   AIC:                             769.9
Df Residuals:                      91   BIC:                             793.3
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.0176      5.347     -2.061      0.042     -21.638      -0.397
CR             1.5014      1.240      1.211      0.229      -0.961       3.964
DA            -5.2206      9.370     -0.557      0.579     -23.834      13.392
FL             3.0104      1.060      2.840      0.006       0.905       5.116
OPM           23.4665      8.076      2.906      0.005       7.425      39.508
RT            -0.0063      0.021     -0.300      0.765      -0.048       0.035
TAT            8.4254      2.351      3.584      0.001       3.756      13.095
DE            -0.7283      2.066     -0.352      0.725      -4.833       3.376
PT            -0.0644      0.096     -0.669      0.505      -0.256       0.127
==============================================================================
Omnibus:                       29.136   Durbin-Watson:                   1.900
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              101.551
Skew:                           0.881   Prob(JB):                     8.88e-23
Kurtosis:                       7.611   Cond. No.                         592.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- S využitím modulu statsmodels byl vytvořen regresní model a zobrazna tabulka se souhrnem významů. 

- Vyhodnocení hodnot koeficientů a výsledků testu statistické významnosti koeficientů. (Pokud je některý koeficient (nebo více koeficientů) nevýznamný, sestavení nového modelu bez tohoto koeficientu (případně beze všech nevýznamných koeficientů).)

In [84]:
# Vyhodnocení testu statistické významnosti

test_tab = pd.DataFrame([
        {"P>|t|": 0.042},
        {"P>|t|": 0.229},
        {"P>|t|": 0.579},
        {"P>|t|": 0.006},
        {"P>|t|": 0.005},
        {"P>|t|": 0.765},
        {"P>|t|": 0.001},
        {"P>|t|": 0.725},
        {"P>|t|": 0.505},
    ],
    index = ["Intercept", "CR", "DA", "FL", "OPM", "RT", "TAT", "DE", "PT"],
)


def evaluate(row):
    coeficient = row["P>|t|"]
    if coeficient < 0.05:
        return "významný"
    else:
        return "nevýznamný"


test_tab["výsledek"] = test_tab.apply(evaluate, axis=1)
test_tab

,P>|t|,výsledek
Intercept,0.042,významný
CR,0.229,nevýznamný
DA,0.579,nevýznamný
FL,0.006,významný
OPM,0.005,významný
RT,0.765,nevýznamný
TAT,0.001,významný
DE,0.725,nevýznamný
PT,0.505,nevýznamný


In [85]:
# sestavení nového regresního modelu bez všech nevýznamných koeficientů

formula_corrig = "Y ~ FL + OPM + TAT"
mod_corrig = smf.ols(formula=formula_corrig, data=data)
res_corrig = mod_corrig.fit()
res_corrig.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.650
Model:                            OLS   Adj. R-squared:                  0.639
Method:                 Least Squares   F-statistic:                     59.51
Date:                Fri, 10 Nov 2023   Prob (F-statistic):           7.99e-22
Time:                        19:58:08   Log-Likelihood:                -377.99
No. Observations:                 100   AIC:                             764.0
Df Residuals:                      96   BIC:                             774.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.4188      2.865     -3.637      0.000     -16.106      -4.732
FL             2.5281      0.208     12.149      0.000       2.115       2.941
OPM           24.8478      7.542      3.295      0.001       9.878      39.818
TAT            8.5518      2.205      3.879      0.000       4.176      12.928
==============================================================================
Omnibus:                       26.748   Durbin-Watson:                   1.921
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               76.291
Skew:                           0.876   Prob(JB):                     2.71e-17
Kurtosis:                       6.904   Cond. No.                         50.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- Pro všechna data odhadni ukazatel Y s využitím tvého modelu a odhadnuté ceny vlož do původní tabulky s daty. 
- Dále vypočítej rozdíl mezi odhadem koeficientu a jeho skutečnou hodnotou. Najdi akcii, kde je tento rozdíl největší (tj. hledáme akcii, které náš model predikuje výrazně vyšší cenu než jaká je ve skutečnosti, tato akcie je potenciálně na trhu podhodnocená).

In [104]:
# přidání odhadnuté ceny
data["Y_estimated"] = res_corrig.fittedvalues

# rozdíl mezi odhadem koeficientu a jeho skutečnou hodnotou
data["residuals"] = res_corrig.resid
data.head()

,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,Sector,Y_estimated,residuals
0,2Q3DU,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93,Industrials,0.502359,1.447641
1,46JNq,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,Communication Services,23.356512,-16.096512
2,4ISOt,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38,Communication Services,6.944718,0.835282
3,5AWwu,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15,Consumer Defensive,15.708512,-9.538512
4,5hbZR,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06,Consumer Cyclical,1.701358,-1.701358


In [107]:
# největší rozdíl -> která akcie je na trhu potenciálně nejvíce podhodnocená

max_resid = data["residuals"].min()
data[data["residuals"] == max_resid]

,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,Sector,Y_estimated,residuals
79,SYXLy,0.76,0.37,18.64,0.0631,12.44,0.41,6.93,6.97,3.4,Industrials,41.779545,-34.809545


Akcii firmy SYXLy model predikuje výrazně vyšší cenu, než jaká ve skutečnosti je, tato akcie je potenciálně na trhu podhodnocená. 

## Bonus 1
- Sestavení modelu s využitím robustní regrese. 
- Vyřazení koeficinetů, které nejsou statisticky významné, a sestavení modelu pouze s významnými koeficienty. 
- Odhady cen vložené do původních dat a opět nalezení potenciálně nejvíce podhodnocenou akcii. Jde v případě robustní regese o stejnou akcii, nebo se akcie liší?

In [108]:
formula = "Y ~ CR + DA + FL + OPM + RT + TAT + DE + PT"
mod_robust = smf.rlm(formula=formula, data=data)
res_robust = mod_robust.fit()
res_robust.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Robust linear Model Regression Results                    
==============================================================================
Dep. Variable:                      Y   No. Observations:                  100
Model:                            RLM   Df Residuals:                       91
Method:                          IRLS   Df Model:                            8
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Sun, 12 Nov 2023                                         
Time:                        20:59:38                                         
No. Iterations:                    37                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.4961      3.453     -3.039      0.002     -17.265      -3.728
CR             2.0316      0.801      2.537      0.011       0.462       3.601
DA            -0.4762      6.052     -0.079      0.937     -12.338      11.386
FL             3.6694      0.685      5.360      0.000       2.328       5.011
OPM           16.3810      5.216      3.140      0.002       6.158      26.604
RT            -0.0029      0.014     -0.216      0.829      -0.029       0.024
TAT            5.2670      1.518      3.469      0.001       2.291       8.243
DE            -2.5507      1.335     -1.911      0.056      -5.166       0.065
PT            -0.0878      0.062     -1.412      0.158      -0.210       0.034
==============================================================================

If the model instance has been used for another fit with different fit parameters, then the fit options might not be the correct ones anymore .
"""

In [109]:
# Vyhodnocení testu statistické významnosti
test_tab_robust = pd.DataFrame([
        {"P>|t|": 0.002},
        {"P>|t|": 0.011},
        {"P>|t|": 0.937},
        {"P>|t|": 0.000},
        {"P>|t|": 0.002},
        {"P>|t|": 0.829},
        {"P>|t|": 0.001},
        {"P>|t|": 0.056},
        {"P>|t|": 0.158},
    ],
    index = ["Intercept", "CR", "DA", "FL", "OPM", "RT", "TAT", "DE", "PT"],
)


test_tab_robust["výsledek"] = test_tab_robust.apply(evaluate, axis=1)
test_tab_robust

,P>|t|,výsledek
Intercept,0.002,významný
CR,0.011,významný
DA,0.937,nevýznamný
FL,0.000,významný
OPM,0.002,významný
RT,0.829,nevýznamný
TAT,0.001,významný
DE,0.056,nevýznamný
PT,0.158,nevýznamný


In [110]:
# sestavení nového robustního regresního modelu bez všech nevýznamných koeficientů

formula_robust_corrig = "Y ~ CR + FL + OPM + TAT"
mod_robust_corrig = smf.rlm(formula=formula_robust_corrig, data=data)
res_robust_corrig = mod_robust_corrig.fit()
res_robust_corrig.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Robust linear Model Regression Results                    
==============================================================================
Dep. Variable:                      Y   No. Observations:                  100
Model:                            RLM   Df Residuals:                       95
Method:                          IRLS   Df Model:                            4
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Sun, 12 Nov 2023                                         
Time:                        21:08:54                                         
No. Iterations:                    23                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -9.9579      1.924     -5.176      0.000     -13.729      -6.187
CR             2.0977      0.681      3.081      0.002       0.763       3.432
FL             2.4422      0.123     19.840      0.000       2.201       2.683
OPM           13.9913      4.560      3.068      0.002       5.054      22.929
TAT            5.3858      1.290      4.176      0.000       2.858       7.913
==============================================================================

If the model instance has been used for another fit with different fit parameters, then the fit options might not be the correct ones anymore .
"""

In [111]:
data["Y_estimated_robust"] = res_robust_corrig.fittedvalues
data["residuals_robust"] = res_robust_corrig.resid
data.head()

,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,Sector,Y_estimated,residuals,Y_estimated_robust,residuals_robust
0,2Q3DU,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93,Industrials,0.502359,1.447641,1.167348,0.782652
1,46JNq,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,Communication Services,23.356512,-16.096512,20.203407,-12.943407
2,4ISOt,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38,Communication Services,6.944718,0.835282,7.832927,-0.052927
3,5AWwu,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15,Consumer Defensive,15.708512,-9.538512,11.557664,-5.387664
4,5hbZR,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06,Consumer Cyclical,1.701358,-1.701358,-0.085471,0.085471


In [112]:
max_resid_robust = data["residuals_robust"].min()
data[data["residuals_robust"] == max_resid_robust]

,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,Sector,Y_estimated,residuals,Y_estimated_robust,residuals_robust
79,SYXLy,0.76,0.37,18.64,0.0631,12.44,0.41,6.93,6.97,3.4,Industrials,41.779545,-34.809545,40.250244,-33.280244


Sestavením modelu s využitím robustní regrese byla nalezena potenciálně nejvíce podhodnocená akcie, a to akcie firmy SYXLy. Jde o stejnou akcii. 

## Bonus 2
- S využitím původního (tedy "nerobustního") modelu je vyčíslena Cookova vzdálenost pro všechny hodnoty. 
- Všechny akcie s Cookovou vzdáleností vyšší než 1 budou vyřazeny a regresní model je přepočítán. 
- Nakonec je opět nalezena potenciálně nejvíce podhodnocená akcie a zjištěno, jestli se jedná o stejnou akcii jako u předchozích modelů.

In [114]:
formula = "Y ~ CR + DA + FL + OPM + RT + TAT + DE + PT"
mod = smf.ols(formula=formula, data=data)
results = mod.fit()
influence = OLSInfluence(results)
data['Cook Distance'] = influence.cooks_distance[0]
data.head()

,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,Sector,Y_estimated,residuals,Y_estimated_robust,residuals_robust,Cook Distance
0,2Q3DU,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93,Industrials,0.502359,1.447641,1.167348,0.782652,0.000046
1,46JNq,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,Communication Services,23.356512,-16.096512,20.203407,-12.943407,0.037578
2,4ISOt,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38,Communication Services,6.944718,0.835282,7.832927,-0.052927,0.000173
3,5AWwu,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15,Consumer Defensive,15.708512,-9.538512,11.557664,-5.387664,0.008283
4,5hbZR,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06,Consumer Cyclical,1.701358,-1.701358,-0.085471,0.085471,0.000082


In [115]:
# vyřazení akcií s Cookovou vzdáleností větší než 1
data_Cook = data[data['Cook Distance'] <= 1]
mod = smf.ols(formula=formula, data=data_Cook)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.660
Method:                 Least Squares   F-statistic:                     24.76
Date:                Sun, 12 Nov 2023   Prob (F-statistic):           1.06e-19
Time:                        21:41:06   Log-Likelihood:                -367.74
No. Observations:                  99   AIC:                             753.5
Df Residuals:                      90   BIC:                             776.8
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.0524      5.364     -1.128      0.262     -16.708       4.604
CR             1.4779      1.186      1.246      0.216      -0.878       3.834
DA           -16.4346      9.678     -1.698      0.093     -35.661       2.792
FL             0.8796      1.228      0.716      0.476      -1.561       3.320
OPM           22.9074      7.728      2.964      0.004       7.555      38.260
RT            -0.0003      0.020     -0.013      0.990      -0.040       0.040
TAT            7.9619      2.254      3.533      0.001       3.484      12.440
DE             4.2726      2.560      1.669      0.099      -0.814       9.359
PT            -0.0192      0.093     -0.206      0.837      -0.204       0.166
==============================================================================
Omnibus:                       20.217   Durbin-Watson:                   1.937
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               53.822
Skew:                           0.649   Prob(JB):                     2.05e-12
Kurtosis:                       6.371   Cond. No.                         640.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [123]:
data["Y_estimated_Cook"] = res.fittedvalues
data["residuals_Cook"] = res.resid
data.head()


,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,Sector,Y_estimated,residuals,Y_estimated_robust,residuals_robust,Cook Distance,Y_estimated_Cook,residuals_Cook
0,2Q3DU,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93,Industrials,0.502359,1.447641,1.167348,0.782652,0.000046,1.033487,0.916513
1,46JNq,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,Communication Services,23.356512,-16.096512,20.203407,-12.943407,0.037578,27.326216,-20.066216
2,4ISOt,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38,Communication Services,6.944718,0.835282,7.832927,-0.052927,0.000173,11.510453,-3.730453
3,5AWwu,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15,Consumer Defensive,15.708512,-9.538512,11.557664,-5.387664,0.008283,14.059466,-7.889466
4,5hbZR,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06,Consumer Cyclical,1.701358,-1.701358,-0.085471,0.085471,0.000082,-1.093074,1.093074


In [124]:
max_resid_Cook = data["residuals_Cook"].min()
data[data["residuals_Cook"] == max_resid_Cook]

,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,Sector,Y_estimated,residuals,Y_estimated_robust,residuals_robust,Cook Distance,Y_estimated_Cook,residuals_Cook
79,SYXLy,0.76,0.37,18.64,0.0631,12.44,0.41,6.93,6.97,3.4,Industrials,41.779545,-34.809545,40.250244,-33.280244,0.575,39.636459,-32.666459


Potenciálně nejvíce podhodnocená akcie je opět vyhodnocena akcie firmy SYXLy

## Bonus 3
- Přidání sloupce "Sektor" (poslední sloupec) jako vysvětlující proměnné s využitím One Hot Encoding. 
- Podívej se, které sektory mají kladný koeficient a které sektory mají záporný koeficient.

In [126]:
field = pd.get_dummies(data["Sector"])
field

,Basic Materials,Communication Services,Conglomerates,Consumer Cyclical,Consumer Defensive,Energy,Financial Services,Healthcare,Industrials,Technology
0,False,False,False,False,False,False,False,False,True,False
1,False,True,False,False,False,False,False,False,False,False
2,False,True,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False
4,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
95,False,False,False,False,False,False,False,True,False,False
96,False,False,False,False,True,False,False,False,False,False
97,False,False,False,False,False,False,False,False,False,True
98,False,False,False,False,False,False,False,False,False,True


In [133]:
data_2 = pd.merge(data, field, left_index=True, right_index=True)
data_2

,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,...,Basic Materials,Communication Services,Conglomerates,Consumer Cyclical,Consumer Defensive,Energy,Financial Services,Healthcare,Industrials,Technology
0,2Q3DU,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93,...,False,False,False,False,False,False,False,False,True,False
1,46JNq,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,...,False,True,False,False,False,False,False,False,False,False
2,4ISOt,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38,...,False,True,False,False,False,False,False,False,False,False
3,5AWwu,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15,...,False,False,False,False,True,False,False,False,False,False
4,5hbZR,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,yhCH3,1.50,0.37,2.33,0.2557,4.92,0.41,0.85,5.18,6.83,...,False,False,False,False,False,False,False,True,False,False
96,YqdX2,0.70,0.27,2.57,0.2363,16.11,0.64,0.69,7.40,2.70,...,False,False,False,False,True,False,False,False,False,False
97,yUjyz,1.49,0.12,2.36,0.2576,8.64,0.51,0.28,5.99,7.59,...,False,False,False,False,False,False,False,False,False,True
98,YX515,2.30,0.64,25.03,0.3758,7.48,0.31,16.08,41.57,10.54,...,False,False,False,False,False,False,False,False,False,True


In [135]:
formula = "Y ~ CR + DA + FL + OPM + RT + TAT + DE + PT + Sector"
mod = smf.ols(formula=formula, data=data_2)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.714
Model:                            OLS   Adj. R-squared:                  0.655
Method:                 Least Squares   F-statistic:                     12.04
Date:                Tue, 14 Nov 2023   Prob (F-statistic):           8.25e-16
Time:                        21:11:27   Log-Likelihood:                -367.95
No. Observations:                 100   AIC:                             771.9
Df Residuals:                      82   BIC:                             818.8
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -7.4799      7.709     -0.970      0.335     -22.816       7.857
Sector[T.Communication Services]    -3.7782      7.099     -0.532      0.596     -17.901      10.344
Sector[T.Conglomerates]             -4.7419     12.308     -0.385      0.701     -29.226      19.742
Sector[T.Consumer Cyclical]          3.0960      7.001      0.442      0.659     -10.831      17.023
Sector[T.Consumer Defensive]        -0.3148      7.466     -0.042      0.966     -15.167      14.537
Sector[T.Energy]                    -6.1282      7.521     -0.815      0.418     -21.089       8.833
Sector[T.Financial Services]        10.5386      9.441      1.116      0.268      -8.243      29.320
Sector[T.Healthcare]                -4.0329      6.689     -0.603      0.548     -17.340       9.275
Sector[T.Industrials]               -7.7802      6.933     -1.122      0.265     -21.572       6.012
Sector[T.Technology]                 1.2953      6.753      0.192      0.848     -12.139      14.730
CR                                   1.3840      1.266      1.093      0.277      -1.134       3.902
DA                                  -2.1773      9.928     -0.219      0.827     -21.926      17.572
FL                                   3.2184      1.120      2.873      0.005       0.990       5.447
OPM                                 17.4452      8.742      1.996      0.049       0.055      34.835
RT                                  -0.0120      0.022     -0.544      0.588      -0.056       0.032
TAT                                  7.0691      2.558      2.763      0.007       1.980      12.158
DE                                  -1.3182      2.175     -0.606      0.546      -5.645       3.009
PT                                  -0.0806      0.104     -0.776      0.440      -0.287       0.126
==============================================================================
Omnibus:                       29.150   Durbin-Watson:                   1.819
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              100.330
Skew:                           0.887   Prob(JB):                     1.64e-22
Kurtosis:                       7.575   Cond. No.                     1.16e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.16e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

poznámka - z nějakého důvodu se v tabulce neobjeví koef. pro Sector[Basic Materials]
-> ověření názvů sloupců
-> ověření, že pro "Basic Materials" máme nějaké hodnoty, tedy True values

In [136]:
data_2.columns

Index(['Company', 'CR', 'DA', 'FL', 'OPM', 'RT', 'TAT', 'DE', 'Y', 'PT',
       'Sector', 'Y_estimated', 'residuals', 'Y_estimated_robust',
       'residuals_robust', 'Cook Distance', 'Y_estimated_Cook',
       'residuals_Cook', 'Basic Materials', 'Communication Services',
       'Conglomerates', 'Consumer Cyclical', 'Consumer Defensive', 'Energy',
       'Financial Services', 'Healthcare', 'Industrials', 'Technology'],
      dtype='object')

In [137]:
basic_mat_rows = data_2[data_2["Basic Materials"] == True]
basic_mat_rows

,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,...,Basic Materials,Communication Services,Conglomerates,Consumer Cyclical,Consumer Defensive,Energy,Financial Services,Healthcare,Industrials,Technology
9,8kTHi,1.11,0.23,2.08,0.1998,8.07,0.56,0.47,1.57,7.26,...,True,False,False,False,False,False,False,False,False,False
38,GRsRd,0.88,0.47,8.48,0.1283,8.48,0.97,3.95,28.55,4.74,...,True,False,False,False,False,False,False,False,False,False
67,pgJHw,0.74,0.18,1.85,0.1619,6.84,0.38,0.33,3.38,5.01,...,True,False,False,False,False,False,False,False,False,False


-> zkusit zadat "Basic Material" přímo:  formula = "Y ~ Basic Materials" selhalo, z důvodu názvu. Kvůli mezeře? Zkusila jsem závorky i uvozovky
-> zkusila jsem přejmenovat na Basic, abych se zbavila mezery, ale selhalo, opět kvůli názvu

Sektor, pro který nebyl koeficient zjištěn: Basic Materials
Sektory, které mají záporný koeficient: Communication Services, Conglomerates, Consumer Defensive, Energy, Healthcare, Industrials
Sektory, které mají kladný koeficient: Consumer Cyclical, Financial Services, Technology

